In [2]:
!wget http://utrdb.ba.itb.cnr.it/utrdb-data/Databases/UTR/data/RefSeq70/3UTRef.Mam.fasta.gz

--2020-08-24 08:31:10--  http://utrdb.ba.itb.cnr.it/utrdb-data/Databases/UTR/data/RefSeq70/3UTRef.Mam.fasta.gz
Resolving utrdb.ba.itb.cnr.it (utrdb.ba.itb.cnr.it)... 150.145.82.18
Connecting to utrdb.ba.itb.cnr.it (utrdb.ba.itb.cnr.it)|150.145.82.18|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 215598607 (206M) [application/x-gzip]
Saving to: ‘3UTRef.Mam.fasta.gz’

3UTRef.Mam.fasta.gz 100%[===================>] 205.61M  16.9MB/s    in 18s     

2020-08-24 08:31:31 (11.5 MB/s) - ‘3UTRef.Mam.fasta.gz’ saved [215598607/215598607]



In [3]:
!gunzip /content/3UTRef.Mam.fasta.gz

In [ ]:
import numpy as np
import random
import pickle


#reading and splitting the data
print('loading the data................................') 
with open('3UTRef.Mam.fasta') as f:
  j=f.read().split('>')
print('batching the data................................') 
#taking a batch in consideration , for RAM issues
j=j[0:100000]  

# function for converting text to numbers a=0 c=1 g=2 t=3

def oh(j):
  x=j.replace('\n','')[26:]
  if set(x) == {'a', 'c', 'g', 't'}:
    d=x.replace('a','1').replace('c','2').replace('g','3').replace('t','4')
    a = np.array(list(map(int, d)))
    #b = np.zeros((a.size, a.max()+1))
    #b[np.arange(a.size),a] = 1
    return a

#running the above functiion and seeing a sample and making three arrays for x and y
print('converting letter to number................................') 
data = list(map(oh, j[1:100000]))
mod_data = list(map(oh, j[1:100000]))
mod_data2 = list(map(oh, j[1:100000]))

#removing None data present from first array
a=1

for j in range(60):
  for i,d in enumerate(data):
    if data[i] is None:
      del data[i]
      a+=1
      #print(type(data[i]))
 

#removing None data present from second array

a=1

for j in range(60):
  for i,d in enumerate(mod_data):
    if mod_data[i] is None:
      del mod_data[i]
      a+=1
      #print(type(mod_data[i]))


a=1

for j in range(60):
  for i,d in enumerate(mod_data2):
    if mod_data2[i] is None:
      del mod_data2[i]
      a+=1
      #print(type(mod_data2[i]))



#randomly changing 10% characters from each example to make input data
print('modifying the data................................') 
for i in range(len(mod_data)):
#for i in range(1):  
  l=int(len(mod_data[i])/2)
  #print(l)
  #for j in range(1):
  for j in range(l):

    mod_data[i][random.randint(0,len(mod_data[i])-1)]=random.choice([1,2,3,4])

#checking if the modification worked
index=3676
fl=False
for i in range(len(data[index])):
  if data[index][i]!=mod_data[index][i]:
    fl=True
if fl:
  print('data successfully modified')

for i in range(len(data)):
  if len(data[i])>3000:
    data[i]=data[i][0:3000]

for i in range(len(mod_data)):
  if len(mod_data[i])>3000:
    mod_data[i]=mod_data[i][0:3000]  

for i in range(len(mod_data2)):
  if len(mod_data2[i])>3000:
    mod_data2[i]=mod_data2[i][0:3000] 


#zero padding
print('padding................................') 
for i in range(len(data)):
  if len(data[i])<3000:
    s = [0] * (3000 - len(data[i]))
    #print(type(data[i]))
    data[i]=list(data[i])+s


for i in range(len(mod_data2)):
  if len(mod_data2[i])<3000:
    s = [0] * (3000 - len(mod_data2[i]))
    #print(type(data[i]))
    mod_data2[i]=list(mod_data[i])+s


for i in range(len(mod_data)):
  if len(mod_data[i])<3000:
    s = [0] * (3000 - len(mod_data[i]))
    #print(type(data[i]))
    mod_data[i]=s+list(mod_data[i])

#one hot function
def one_hot_con(data):
  a = np.array(data)
  b = np.zeros((a.size, 5))
  b[np.arange(a.size),a] = 1
  return b

#making final processed dataset after converting one hot
print('preparing the final data................................') 
X1 =np.array(list(map(one_hot_con, mod_data)))
X2 =np.array(list(map(one_hot_con, mod_data2)))
Y = np.array(list(map(one_hot_con, data)))


print('pickeling................................') 
with open('X1.data', 'wb') as fh:
   pickle.dump(X1, fh)

with open('X2.data', 'wb') as fh:
   pickle.dump(X2, fh)

with open('Y.data', 'wb') as fh:
   pickle.dump(Y, fh)  

print('Successfully processed')       

loading the data................................
batching the data................................
converting letter to number................................
modifying the data................................
data successfully modified
padding................................


MODEL CREATION , TRAINING , AND SAVING

In [ ]:
from random import randint
from numpy import array
from numpy import argmax
from numpy import array_equal
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow import keras
import pickle


# returns train, inference_encoder and inference_decoder models
def define_models():
	# define training encoder
	encoder_inputs = Input(shape=(3000,5))
	encoder = LSTM(512, return_state=True)
	encoder_outputs, state_h, state_c = encoder(encoder_inputs)
	encoder_states = [state_h, state_c]
	# define training decoder
	decoder_inputs = Input(shape=(3000,5))
	decoder_lstm = LSTM(512, return_sequences=True, return_state=True)
	decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
	decoder_dense = Dense(5, activation='softmax')
	decoder_outputs = decoder_dense(decoder_outputs)
	model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
	# define inference encoder
	encoder_model = Model(encoder_inputs, encoder_states)
	# define inference decoder
	decoder_state_input_h = Input(shape=(512,))
	decoder_state_input_c = Input(shape=(512,))
	decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
	decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
	decoder_states = [state_h, state_c]
	decoder_outputs = decoder_dense(decoder_outputs)
	decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)
	# return all models
	return model, encoder_model, decoder_model

#for loading the pickle in case of training later

'''
print('loading X1..........')
pickle_x1 = open ("X1.data", "rb")
X1 = pickle.load(pickle_x1)
print('loading X2..........')
pickle_x2 = open ("X2.data", "rb")
X2 = pickle.load(pickle_x2)
print('loading Y..........')
pickle_y = open ("Y.data", "rb")
Y = pickle.load(pickle_y)
'''


print('splitting the data.................')
x1_train=X1[0:90000]
x2_train=X2[0:90000]
y_train=Y[0:90000]
x1_test=X1[90000:]
x2_test=X2[90000:]
y_test=Y[90000:]


print('creating model..........')
model, encoder_model, decoder_model=define_models()
model.summary()
print('compiling model..........')
opt = keras.optimizers.Adam(learning_rate=0.0001,clipnorm=1.0)
model.compile(loss='categorical_crossentropy', optimizer=opt,		metrics=['accuracy'])
print('starting the training..........')
model.fit(
    x=[x1_train,x2_train],
    y=y_train,
    batch_size=32,
		validation_data=([x1_test,x2_test],y_test),
    epochs=100,
    shuffle=True,
)

In [ ]:
model.save('model.h5')
encoder_model.save('encoder.h5')
decoder_model.save('decoder.h5')

In [ ]:
import keras
import numpy as np
print('loading model.........................')
model = keras.models.load_model('model.h5')

loading model.........................


In [ ]:
import numpy as np

#sequence length 
seqlen=3000

#insert your sequence here
seq='aaaaaaaa'

print('replacing letters with numbers.........................')
if set(seq) == {'a', 'c', 'g', 't'}:
  d=seq.replace('a','1').replace('c','2').replace('g','3').replace('t','4')
  a1 = np.array(list(map(int, d)))
  a2 = np.array(list(map(int, d)))
  print(a1)


print('zero padding.........................')
if len(a1)<seqlen:
  s = [0] * (seqlen - len(a1))
  a1=s+list(a1)
#print(a1)


if len(a2)<seqlen:
  s = [0] * (seqlen - len(a2))
  a2=list(a2)+s
#print(a2)

print('converting to one hot.........................')
def one_hot_con(data):
  a = np.array(data)
  b = np.zeros((a.size, 5))
  b[np.arange(a.size),a] = 1
  return b

x1=one_hot_con(a1)
#print(x1)

x2=one_hot_con(a2)
#print(x2)





print('calculating output.........................')
out_r=model.predict([x1.reshape(1,seqlen,5),x2.reshape(1,seqlen,5)])
out_l=[]
out=[]
for i in range(len(seq)):
  out.append(out_r[0][i])
  out_l.append(list(out_r[0][i]))

print('\n')
print('\n')
print('probablity distribution')
print(out_l)
print('first number in each array denotes the probablity of the network being not sure')

print('\n')
print('\n')
print('\n')
ls=list(np.array(out).argmax(axis=1))
st=''
for i in ls:
  st+=str(i)
print('corrected sequence')
print(seq)
print(st.replace('0','x').replace('1','a').replace('2','c').replace('3','g').replace('4','t'))


print('x : not sure')
